In [2]:
import nltk
import sklearn
import pandas as pd
import gensim
import FuzzyTM
import re

In [3]:
articles = pd.read_csv('../data/us_equities_news_dataset.csv')
articles = articles[articles['content'].notna()]
articles

,id,ticker,title,category,content,release_date,provider,url,article_id
0,221515,NIO,Why Shares of Chinese Electric Car Maker NIO A...,news,What s happening\nShares of Chinese electric c...,2020-01-15,The Motley Fool,https://invst.ly/pigqi,2060327
1,221516,NIO,NIO only consumer gainer Workhorse Group amon...,news,Gainers NIO NYSE NIO 7 \nLosers MGP Ingr...,2020-01-18,Seeking Alpha,https://invst.ly/pje9c,2062196
2,221517,NIO,NIO leads consumer gainers Beyond Meat and Ma...,news,Gainers NIO NYSE NIO 14 Village Farms In...,2020-01-15,Seeking Alpha,https://invst.ly/pifmv,2060249
3,221518,NIO,NIO NVAX among premarket gainers,news,Cemtrex NASDAQ CETX 85 after FY results \n...,2020-01-15,Seeking Alpha,https://invst.ly/picu8,2060039
4,221519,NIO,PLUG NIO among premarket gainers,news,aTyr Pharma NASDAQ LIFE 63 on Kyorin Pharm...,2020-01-06,Seeking Alpha,https://seekingalpha.com/news/3529772-plug-nio...,2053096
...,...,...,...,...,...,...,...,...,...
221508,443024,T,Crude And Steel Still In Sync,opinion,We have been reporting on the trade off betwee...,2012-10-04,Ivan Kitov,https://www.investing.com/analysis/crude-and-s...,138733
221509,443025,T,Forget AT T This Is The Telecom Stock You Sho...,opinion,It s the largest cell phone provider in the wo...,2012-05-30,StreetAuthority,https://www.investing.com/analysis/forget-at-t...,124829
221510,443026,T,Wall Street Exposed Part 3 How Dividends C...,opinion,Before we dicuss how the mechanism of dividend...,2012-07-16,Portfolio Cafe,https://www.investing.com/analysis/wall-street...,129651
221511,443027,T,Weighing The Week Ahead It s All About Jobs,opinion,From start to finish the coming week will hav...,2012-09-02,Jeff Miller,https://www.investing.com/analysis/weighing-th...,134926


# Descriptive statistics
- How many documents
- How many words
- How much lexical variation in the texts
- Average sentence length
- Average document length
- 100 most common words
- 20 most indicative words per class
- Dispersion plot

In [4]:
#number of documents
nr_of_docu = articles['id'].nunique()

In [5]:
# Count words in each row
articles['word_count'] = articles['content'].apply(lambda x: len(x.split()))

#Sum the number of words of each row
total_word_count = articles['word_count'].sum()

In [6]:
# Combine all text from the 'content' column into a single string
combined_text = ' '.join(articles['content'])

In [7]:
#Create a set of the words, these are the unique words (these are split since it gives a memory error)

unique_words = set()

# Tokenize and count unique words in smaller chunks
chunk_size = 1000  # Adjust this value based on your available memory and DataFrame size

for i in range(0, len(articles), chunk_size):
    chunk = articles.loc[i:i+chunk_size-1]  # Get a chunk of the DataFrame
    combined_text = ' '.join(chunk['content'])  # Combine text from the chunk
    chunk_words = combined_text.split()  # Tokenize the chunk
    unique_words.update(chunk_words)  # Add unique words to the set

# Count the total number of unique words
unique_word_count = len(unique_words)

In [8]:
lexical_variation = unique_word_count/total_word_count

Number of documents: 221505
Total number of words: 131445300
Number unique tokens: 400076
Lexical variation:  0.0030436691155940913


In [10]:
#Average document length
avg_doc_length = total_word_count/nr_of_docu

In [12]:
def count_sentences(text):
    sentences = re.split(r'[\n]+|[ ]{2,}(?=[A-Z][a-z])', text)
    return len(sentences)

# Apply the count_sentences function to the 'content' column and save the counts in a new column
articles['sentence_count'] = articles['content'].apply(count_sentences)

In [21]:
nr_of_sentences = articles['sentence_count'].sum()
avg_sentence_length = total_word_count/nr_of_sentences

In [22]:
avg_doc_length_sentence = nr_of_sentences/nr_of_docu

In [29]:
print("Number of documents:", nr_of_docu)
print("Total number of words:", total_word_count)
print("Number unique words:", unique_word_count)
print("Lexical variation: ", round(lexical_variation, 5))
print("Average sentence length is ", round(avg_sentence_length,2), "Words")
print("Average document length ", round(avg_doc_length,2), "Words")
print("Average document length ", round(avg_doc_length_sentence,2), "Sentences")

Number of documents: 221505
Total number of words: 131445300
Number unique words: 400076
Lexical variation:  0.00304
Average sentence length is  17.5 Words
Average document length  593.42 Words
Average document length  33.9 Sentences
